In [1]:
# start the Spark Context
import findspark
findspark.init()

In [2]:
import pyspark # only run after findspark.init()

In [3]:
from pyspark.sql import SparkSession
sc = pyspark.SparkContext(appName="col")

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [6]:
data = [('patty', 'spring', 'baseball', 64),
        ('matty', 'autumn', 'hockey', 90),
        ('cathy', 'spring', 'baseball', 100),
        ('sandy', 'autumn', 'soccer', 50),
        ('joey', 'summer', 'soccer', 73),
        ('tammy', 'spring', 'soccer', 86),
        ('marley', 'autumn', 'hockey', 100)]

In [7]:
# create an rdd
rdd = sc.parallelize(data)

In [8]:
# take first 4 elements
rdd.take(4)


[('patty', 'spring', 'baseball', 64),
 ('matty', 'autumn', 'hockey', 90),
 ('cathy', 'spring', 'baseball', 100),
 ('sandy', 'autumn', 'soccer', 50)]

In [9]:
# create a dataframe from an rdd and name the columns
df = spark.createDataFrame(rdd, ['player', 'season', 'sport', 'ranking'])

In [10]:
df.show()

+------+------+--------+-------+
|player|season|   sport|ranking|
+------+------+--------+-------+
| patty|spring|baseball|     64|
| matty|autumn|  hockey|     90|
| cathy|spring|baseball|    100|
| sandy|autumn|  soccer|     50|
|  joey|summer|  soccer|     73|
| tammy|spring|  soccer|     86|
|marley|autumn|  hockey|    100|
+------+------+--------+-------+



In [11]:
# show the first 4 elements
df.show(4)

+------+------+--------+-------+
|player|season|   sport|ranking|
+------+------+--------+-------+
| patty|spring|baseball|     64|
| matty|autumn|  hockey|     90|
| cathy|spring|baseball|    100|
| sandy|autumn|  soccer|     50|
+------+------+--------+-------+
only showing top 4 rows



In [12]:
# take the header of the dataframe
df.head()

Row(player='patty', season='spring', sport='baseball', ranking=64)

In [13]:
# count the number of elements in the dataframe
df.count()

7

In [14]:
# describe the dataframe
df.describe().show()


+-------+------+------+--------+------------------+
|summary|player|season|   sport|           ranking|
+-------+------+------+--------+------------------+
|  count|     7|     7|       7|                 7|
|   mean|  null|  null|    null| 80.42857142857143|
| stddev|  null|  null|    null|18.884359867865463|
|    min| cathy|autumn|baseball|                50|
|    max| tammy|summer|  soccer|               100|
+-------+------+------+--------+------------------+



In [15]:
# print schema
df.printSchema()

root
 |-- player: string (nullable = true)
 |-- season: string (nullable = true)
 |-- sport: string (nullable = true)
 |-- ranking: long (nullable = true)



In [16]:
# select columns
df.select(['player', 'sport', 'ranking']).show()

+------+--------+-------+
|player|   sport|ranking|
+------+--------+-------+
| patty|baseball|     64|
| matty|  hockey|     90|
| cathy|baseball|    100|
| sandy|  soccer|     50|
|  joey|  soccer|     73|
| tammy|  soccer|     86|
|marley|  hockey|    100|
+------+--------+-------+



In [17]:
# filter where values are greater than 85 in rows of column 'ranking'
df.filter(df['ranking'] > 85).show()

+------+------+--------+-------+
|player|season|   sport|ranking|
+------+------+--------+-------+
| matty|autumn|  hockey|     90|
| cathy|spring|baseball|    100|
| tammy|spring|  soccer|     86|
|marley|autumn|  hockey|    100|
+------+------+--------+-------+



In [18]:
# filter where the season is autumn
df.filter(df['season'] == 'autumn').show()


+------+------+------+-------+
|player|season| sport|ranking|
+------+------+------+-------+
| matty|autumn|hockey|     90|
| sandy|autumn|soccer|     50|
|marley|autumn|hockey|    100|
+------+------+------+-------+



In [19]:
# select columns and create a new one
df.select(df['player'], df['season'], df['sport'], df['ranking'],
          (df['ranking'] - 15).alias('new_ranking')).show()

+------+------+--------+-------+-----------+
|player|season|   sport|ranking|new_ranking|
+------+------+--------+-------+-----------+
| patty|spring|baseball|     64|         49|
| matty|autumn|  hockey|     90|         75|
| cathy|spring|baseball|    100|         85|
| sandy|autumn|  soccer|     50|         35|
|  joey|summer|  soccer|     73|         58|
| tammy|spring|  soccer|     86|         71|
|marley|autumn|  hockey|    100|         85|
+------+------+--------+-------+-----------+



In [20]:
# use withColumn() to create a new column
df.withColumn('lowered_ranking', df['ranking']* 0.33).show()

+------+------+--------+-------+------------------+
|player|season|   sport|ranking|   lowered_ranking|
+------+------+--------+-------+------------------+
| patty|spring|baseball|     64|             21.12|
| matty|autumn|  hockey|     90|29.700000000000003|
| cathy|spring|baseball|    100|              33.0|
| sandy|autumn|  soccer|     50|              16.5|
|  joey|summer|  soccer|     73|             24.09|
| tammy|spring|  soccer|     86|28.380000000000003|
|marley|autumn|  hockey|    100|              33.0|
+------+------+--------+-------+------------------+



In [21]:
# sort values in column
df.sort(df['ranking']).show()

+------+------+--------+-------+
|player|season|   sport|ranking|
+------+------+--------+-------+
| sandy|autumn|  soccer|     50|
| patty|spring|baseball|     64|
|  joey|summer|  soccer|     73|
| tammy|spring|  soccer|     86|
| matty|autumn|  hockey|     90|
|marley|autumn|  hockey|    100|
| cathy|spring|baseball|    100|
+------+------+--------+-------+



In [22]:
# sort values in column in descending order
df.sort(df['ranking'].desc()).show()

+------+------+--------+-------+
|player|season|   sport|ranking|
+------+------+--------+-------+
|marley|autumn|  hockey|    100|
| cathy|spring|baseball|    100|
| matty|autumn|  hockey|     90|
| tammy|spring|  soccer|     86|
|  joey|summer|  soccer|     73|
| patty|spring|baseball|     64|
| sandy|autumn|  soccer|     50|
+------+------+--------+-------+



In [23]:
# a different way to write the same code as previous
df.sort(df.ranking.desc()).show()  

+------+------+--------+-------+
|player|season|   sport|ranking|
+------+------+--------+-------+
|marley|autumn|  hockey|    100|
| cathy|spring|baseball|    100|
| matty|autumn|  hockey|     90|
| tammy|spring|  soccer|     86|
|  joey|summer|  soccer|     73|
| patty|spring|baseball|     64|
| sandy|autumn|  soccer|     50|
+------+------+--------+-------+

